<a href="https://colab.research.google.com/github/sangeetsaurabh/tweet_sentiment_extraction/blob/master/transform_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

##### Mount the Google drive for input/output

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_folder = "/content/drive/My Drive/tweet-sentiment-extraction/data"
tmp_folder = '/tmp'

In [ ]:
#### Copy the dataset
train_data = pd.read_csv(data_folder + "/train.csv")
test_data = pd.read_csv(data_folder + "/test.csv")

#### copying the original data just to see what original text was as we update the data
train_data["old_text"] = train_data.text
train_data["old_sel_text"] = train_data.selected_text

test_data["old_text"] = test_data.text

### Drop the data with null value
train_data = train_data[train_data.text == train_data.text]

In [ ]:
train_data.head()

,textID,text,selected_text,sentiment,old_text,old_sel_text
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"I`d have responded, if I were going","I`d have responded, if I were going"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD
2,088c60f138,my boss is bullying me...,bullying me,negative,my boss is bullying me...,bullying me
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview! leave me alone,leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"Sons of ****, why couldn`t they put them on t...","Sons of ****,"


In [ ]:
#### clean up following characters
pd.set_option("display.max_colwidth", None)

### impacting 1 sentence
train_data['text'] = train_data['text'].str.replace('\xa0',' ')
train_data['selected_text'] = train_data['selected_text'].str.replace('\xa0',' ')


### Impacing 2 sentences
train_data['text'] = train_data['text'].str.replace('Â',' ')
train_data['selected_text'] = train_data['selected_text'].str.replace('Â',' ')

### Impacing 5 sentences
train_data['text'] = train_data['text'].str.replace('``',' ')
train_data['selected_text'] = train_data['selected_text'].str.replace('``',' ')

### Impacing 1 sentence
train_data['text'] = train_data['text'].str.replace('´','`')
train_data['selected_text'] = train_data['selected_text'].str.replace('´','`')

### Impacing 1 sentence
train_data['text'] = train_data['text'].str.replace('\t','')
train_data['selected_text'] = train_data['selected_text'].str.replace('\t','')

### Impacing 155 sentences
train_data['text'] = train_data['text'].str.replace('ï¿½',' ')
train_data['selected_text'] = train_data['selected_text'].str.replace('ï¿½',' ')

###Impacting 1220 rows (only 5 of them are in selected_text) (removing all the URLs)
train_data['text'] = train_data['text'].str.replace(r'http\S+', 'URL', regex=True)
train_data['selected_text'] = train_data['selected_text'].str.replace(r'http\S+', 'URL', regex=True)

#### Impacting 2 sentects. It seems to be a bug in the data, text doesn't have those characters 
train_data['selected_text'] = train_data['selected_text'].str.replace(r'ï', '', regex=True)

#### Impacting one sentence (no impact on test data, so just ok to remove it)
####Impacting 8 sentences. Most of them are neutral. Generally if a sentence has this, include in selected text
train_data['text'] = train_data['text'].str.replace('\(:\|', '')
train_data['selected_text'] = train_data['selected_text'].str.replace('\(:\|', '')


####Impacting 8 sentences. Most of them are neutral. Generally if a sentence has this, include in selected text
train_data['text'] = train_data['text'].str.replace(':\|', '')
train_data['text'] = train_data['text'].str.replace(':-\|', '')

train_data['selected_text'] = train_data['selected_text'].str.replace(':\|', '')
train_data['selected_text'] = train_data['selected_text'].str.replace(':-\|', '')

 

#### Remove some of the special characters emojis (impacts 5 to 7 rows)
train_data['text'] = train_data['text'].str.replace(':\*\*-\(', '')
train_data['text'] = train_data['text'].str.replace(':\*\*\(', '')
train_data['text'] = train_data['text'].str.replace(':\*', '')

train_data['selected_text'] = train_data['selected_text'].str.replace(':\*\*-\(', '')
train_data['selected_text'] = train_data['selected_text'].str.replace(':\*\*\(', '')
train_data['selected_text'] = train_data['selected_text'].str.replace(':\*', '')

####Impacting 20 sentences. Most of them are neutral. Generally if a sentence has this, include in selected text
train_data['text'] = train_data['text'].str.replace(':O', '')
train_data['selected_text'] = train_data['selected_text'].str.replace(':O', '')




###Remove leading and trailing spaces
train_data['text'] = train_data['text'].str.strip()
train_data['selected_text'] = train_data['selected_text'].str.strip()

train_data['text'] = train_data['text'].str.replace(r'\s{2,}', ' ',regex=True)
train_data['selected_text'] = train_data['selected_text'].str.replace(r'\s{2,}', ' ',regex=True)





### Impacing 1 sentence
test_data['text'] = test_data['text'].str.replace('Â',' ')

### Impacing 1 sentence
test_data['text'] = test_data['text'].str.replace('``',' ')


### Impacing 1 sentence
test_data['text'] = test_data['text'].str.replace('\t',' ')

### Impacing 13 sentences
test_data['text'] = test_data['text'].str.replace('ï¿½',' ')

### Impacting 174 rows (removing all the URLs)
test_data['text'] = test_data['text'].str.replace(r'http\S+', 'URL', regex=True)

### Impacting 2 rows. Just include this in the output
test_data['text'] = test_data['text'].str.replace(':\|', '')
test_data['text'] = test_data['text'].str.replace(':-\|', '')


### Impacting 1 row. Just include this in the output
test_data['text'] = test_data['text'].str.replace('¡', '')

#### Remove some of the special characters emojis (impacts 1 row)
test_data['text'] = test_data['text'].str.replace(':\*', '')

#### Remove some of the special characters emojis (impacts 3 rows)
test_data['text'] = test_data['text'].str.replace(':O', '')


###Remove leading, trailing and duplicate spaces
test_data['text'] = test_data['text'].str.strip()
test_data['text'] = test_data['text'].str.replace(r'\s{2,}', ' ',regex=True)






In [ ]:
### Some words are partially copied in selected text. This function identifies such word and updates the selected text.
def update_partial_token(row):
  return_row = row["sel_text_token"].copy()
  if len(row["set_diff"]) > 0:
    for item in row["set_diff"]:
      actual_text = [s for s in row["text_token"] if item in s]
      if len(actual_text) == 1:
        sel_txt_indx = return_row.index(item)
        return_row[sel_txt_indx] = actual_text[0]
      else:
        return_row = list(filter((item).__ne__,return_row))
  return return_row

        
    


In [ ]:
sub = ">"
#test_data["Indexes"]= test_data["text"].str.find(sub)
#X = test_data[(test_data.Indexes != -1)]
#sub =  r"[\w'`]+|[.,!?;\^_={}\(\)*:<>\d]+"
sub =  r"[\w]+|[\d.#$%&+-/@,!?;\^_={}\(\)*:<>\d]+"

#X["text_token"]= X["text"].str.findall(sub)

#X[(X.Indexes != -1)]


In [ ]:
### drop these characters {'\t', '\xa0', '´', '̂'}

sub =  r"[\w'`]+|[\d.#$%&+-/@,!?;\^_={}\(\)*:<>\d]+"

train_data["text"] = train_data["text"].str.lower()
train_data["selected_text"] = train_data["selected_text"].str.lower()

#### creating the token for text separating punctuation and alpha
train_data["text_token"]= train_data["text"].str.findall(sub)
train_data["sel_text_token"]= train_data["selected_text"].str.findall(sub)

#### Many of the words are copied incomplete into selected text. Finding out all the scenarios when tokens are different
train_data["set_diff"] = train_data["sel_text_token"].apply(set) - train_data["text_token"].apply(set)

train_data["final_sel_text_token"] = train_data.apply(update_partial_token,axis=1)

#### Correct some of the rows manually
train_data.loc[11808,'final_sel_text_token'] = ['hate']
train_data.loc[14172,'final_sel_text_token'] = ['bunny']
train_data.loc[24929,'final_sel_text_token'] = ['wow']
train_data.loc[25691,'final_sel_text_token'] = ['cheshire_cat_']

train_data["final_set_diff"] = train_data["final_sel_text_token"].apply(set) - train_data["text_token"].apply(set)

train_data[["text","text_token","selected_text","sel_text_token","set_diff","final_set_diff","final_sel_text_token"]].tail(10)

,text,text_token,selected_text,sel_text_token,set_diff,final_set_diff,final_sel_text_token
27471,"i`m defying gravity. and nobody in alll of oz, no wizard that there is or was, is ever gonna bring me down","[i`m, defying, gravity, ., and, nobody, in, alll, of, oz, ,, no, wizard, that, there, is, or, was, ,, is, ever, gonna, bring, me, down]","i`m defying gravity. and nobody in alll of oz, no wizard that there is or was, is ever gonna bring me down","[i`m, defying, gravity, ., and, nobody, in, alll, of, oz, ,, no, wizard, that, there, is, or, was, ,, is, ever, gonna, bring, me, down]",{},{},"[i`m, defying, gravity, ., and, nobody, in, alll, of, oz, ,, no, wizard, that, there, is, or, was, ,, is, ever, gonna, bring, me, down]"
27472,url - wanted to visit the animals but we were too late,"[url, -, wanted, to, visit, the, animals, but, we, were, too, late]",were too late,"[were, too, late]",{},{},"[were, too, late]"
27473,in spoke to you yesterday and u didnt respond girl wassup though!,"[in, spoke, to, you, yesterday, and, u, didnt, respond, girl, wassup, though, !]",in spoke to you yesterday and u didnt respond girl wassup though!,"[in, spoke, to, you, yesterday, and, u, didnt, respond, girl, wassup, though, !]",{},{},"[in, spoke, to, you, yesterday, and, u, didnt, respond, girl, wassup, though, !]"
27474,so i get up early and i feel good about the day. i walk to work and i`m feeling alright. but guess what... i don`t work today.,"[so, i, get, up, early, and, i, feel, good, about, the, day, ., i, walk, to, work, and, i`m, feeling, alright, ., but, guess, what, ..., i, don`t, work, today, .]",i feel good ab,"[i, feel, good, ab]",{ab},{},"[i, feel, good, about]"
27475,enjoy ur night,"[enjoy, ur, night]",enjoy,[enjoy],{},{},[enjoy]
27476,wish we could come see u on denver husband lost his job and can`t afford it,"[wish, we, could, come, see, u, on, denver, husband, lost, his, job, and, can`t, afford, it]",d lost,"[d, lost]",{d},{},[lost]
27477,"i`ve wondered about rake to. the client has made it clear .net only, don`t force devs to learn a new lang #agile #ccnet","[i`ve, wondered, about, rake, to, ., the, client, has, made, it, clear, ., net, only, ,, don`t, force, devs, to, learn, a, new, lang, #, agile, #, ccnet]",", don`t force","[,, don`t, force]",{},{},"[,, don`t, force]"
27478,yay good for both of you. enjoy the break - you probably need it after such hectic weekend take care hun xxxx,"[yay, good, for, both, of, you, ., enjoy, the, break, -, you, probably, need, it, after, such, hectic, weekend, take, care, hun, xxxx]",yay good for both of you.,"[yay, good, for, both, of, you, .]",{},{},"[yay, good, for, both, of, you, .]"
27479,but it was worth it ****.,"[but, it, was, worth, it, ****.]",but it was worth it ****.,"[but, it, was, worth, it, ****.]",{},{},"[but, it, was, worth, it, ****.]"
27480,all this flirting going on - the atg smiles. yay. ((hugs)),"[all, this, flirting, going, on, -, the, atg, smiles, ., yay, ., ((, hugs, ))]",all this flirting going on - the atg smiles. yay. ((hugs),"[all, this, flirting, going, on, -, the, atg, smiles, ., yay, ., ((, hugs, )]",{)},{},"[all, this, flirting, going, on, -, the, atg, smiles, ., yay, ., ((, hugs, ))]"


In [ ]:
train_data[train_data['final_set_diff'].apply(len) > 0]

,textID,text,selected_text,sentiment,old_text,old_sel_text,text_token,sel_text_token,set_diff,final_sel_text_token,final_set_diff


So the above made sure that there is no difference in token between of selected text and text.

#### Difference in Test tokens and Training Tokens

In [ ]:
pd.set_option("display.max_rows",500)

sub =  r"[\w]+|[\d.#$%&+-/@,!?;\^_={}\(\)*:<>\d]+"
test_data["text"] = test_data["text"].str.lower()
test_data["text_token"]= test_data["text"].str.findall(sub)

test_data.head()

,textID,text,sentiment,old_text,text_token
0,f87dea47db,last session of the day url,neutral,Last session of the day http://twitpic.com/67ezh,"[last, session, of, the, day, url]"
1,96d74cb729,shanghai is also really exciting (precisely -- skyscrapers galore). good tweeps in china: (sh) (bj).,positive,Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China: (SH) (BJ).,"[shanghai, is, also, really, exciting, (, precisely, --, skyscrapers, galore, )., good, tweeps, in, china, :, (, sh, ), (, bj, ).]"
2,eee518ae67,"recession hit veronique branquinho, she has to quit her company, such a shame!",negative,"Recession hit Veronique Branquinho, she has to quit her company, such a shame!","[recession, hit, veronique, branquinho, ,, she, has, to, quit, her, company, ,, such, a, shame, !]"
3,01082688c6,happy bday!,positive,happy bday!,"[happy, bday, !]"
4,33987a8ee5,url - i like it!!,positive,http://twitpic.com/4w75p - I like it!!,"[url, -, i, like, it, !!]"


In [ ]:
test_train_diff = set(test_data["text_token"].sum()) - set(train_data["text_token"].sum())

In [ ]:
len(test_train_diff)

1929

So, there are 1929 words in Test tweets that's not there in training set.

In [ ]:
test_train_diff

{'lowkey',
 'hedro',
 'aarg',
 'impersonator',
 'cuzin',
 'tunage',
 'judgements',
 'durrin',
 'apostro',
 't0g0',
 'phir',
 'ssn',
 'qc',
 '_adel',
 'loos',
 'suuuppper',
 'interior',
 'tda',
 'scattegories',
 'prospects',
 'customized',
 'barrels',
 'sorronda',
 'higherscore',
 'sirs',
 'grandaughter',
 'shadez',
 'scr',
 'jolla',
 'downwards',
 'embarrass',
 'weapons',
 'trafficjam',
 'reconsituted',
 'woohooo',
 'weirton',
 'butmy',
 'margeritas',
 'ruddy',
 'slaughter',
 'hignfy',
 'soccergame',
 '&5',
 'hypocrite',
 'stickers',
 'biteeee',
 'alist',
 'medicated',
 'highs',
 'breton',
 'hyperactive',
 'casually',
 'viewable',
 '_ember',
 '30s',
 'wontt',
 '$20.00',
 'chrystina',
 'zimmermann',
 'greeeeeeaaaaatttttttt',
 'joel',
 'watcing',
 '#6.',
 'classs',
 'controller',
 'bork',
 'mommyyy',
 '981',
 'guerilla',
 '_hameron',
 'didddd',
 'aldi',
 'elegant',
 'dhellohannah5',
 'bouvier',
 'snapp',
 'mixture',
 'mondayyy',
 'grammer',
 'sleeeepp',
 'testrun',
 'everest',
 'hearsay'

A lot of words are different because they are misspelled or are alphanumeric.

#### Lemmetization

Let's lemmetize them to see if that helps

In [ ]:
import spacy
nlp = spacy.load('en')

In [ ]:
unique_train_words = list(set(train_data["text_token"].sum()))
unique_lemma_train_words = []
separator = ', '
doc = nlp(separator.join(unique_train_words))
for token in doc:
  unique_lemma_train_words.append(token.lemma_)



In [ ]:
print (f'The number of unique words in train dataset after lemmetization {len(unique_lemma_train_words)}')

The number of unique words in train dataset after lemmetization 58904


In [ ]:
unique_test_words = list(set(test_data["text_token"].sum()))
unique_lemma_test_words = []
separator = ', '
doc = nlp(separator.join(unique_test_words))
for token in doc:
  unique_lemma_test_words.append(token.lemma_)

In [ ]:
print (f'The number of unique words in train dataset after lemmetization {len(unique_lemma_test_words)}')

The number of unique words in train dataset after lemmetization 15038


In [ ]:
test_train_diff = set(unique_lemma_test_words) - set(unique_lemma_train_words)
len(test_train_diff)

1692

Still there are 1692 words that are different.

#### Spellchecker

Let's see if fixing the spelling may help

In [ ]:
!pip install pyspellchecker

     |████████████████████████████████| 1.9MB 2.6MB/s 


In [ ]:
from spellchecker import SpellChecker
spell = SpellChecker()
misspelled = spell.unknown(test_train_diff)

In [ ]:
spell_corrected_word = []

word_count = 0
for word in misspelled:
  spell_corrected_word.append(spell.correction(word))
  #print(word, spell.correction(word))
  #print(spell.candidates(word))
  #word_count += 1
  #if word_count > 10:
  #  break

In [ ]:
len(set(spell_corrected_word) - set(unique_lemma_train_words))

523

After spelling correction, there are 523 words that are different.

In [ ]:
set(spell_corrected_word) - set(unique_lemma_train_words)

{'--------',
 '-----------',
 '-___-',
 '..............',
 '1970s',
 '30/6:45-',
 '5000th',
 '^)<^',
 'a209',
 'aaaaaaaa',
 'abou',
 'absorb',
 'after-school',
 'agatha',
 'ahahahahahahahahah',
 'air-conditioned',
 'aksum',
 'alfradique',
 'aloft',
 'amass',
 'annandale',
 'anode',
 'apostle',
 'appin',
 'applecart',
 'are',
 'arghhhhhhhhh',
 'arose',
 'ashleyyyy',
 'aura',
 'b2',
 'babylon',
 'baku',
 'banknote',
 'barro',
 'based',
 'battens',
 'bawling',
 'bbs',
 'been',
 'bells',
 'billyidol',
 'biteeee',
 'blerrrrrrrrrrgh',
 'blockkkkk',
 'blogtalk',
 'bloooooow',
 'boardgame',
 'bolan',
 'bolton',
 'boogie',
 'bookcase',
 'boooooooooo',
 'borland',
 'bostock',
 'bothy',
 'boto',
 'boxes',
 'braddddddddd',
 'branquinho',
 'breakupword',
 'brittneyy',
 'brrrr',
 'bumface',
 'bupa',
 'bushy',
 'buttressing',
 'butty',
 'caixa',
 'caladesi',
 'camiguin',
 'canadamigo',
 'canton',
 'carelessness',
 'cassoulet',
 'catacombs',
 'cerezo',
 'chaayaa',
 'chandra',
 'charon',
 'checks',
 'c

More experiments when we get to build the model. Good to understand these information about the text tokens.